In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libcudnn8 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '8.1.0.77-1+cuda11.2' for 'libcudnn8' was not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.0 MB/s eta 0:00:00


In [ ]:
import logging
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = 'drive/MyDrive/shk_to_en_tokenizers'
# tf.keras.utils.get_file(
#     f'{model_name}.zip',
#     f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
#     cache_dir='.', cache_subdir='', extract=True
# )
tokenizers = tf.saved_model.load(model_name)

In [ ]:
df = pd.read_csv('drive/MyDrive/final.csv')
# x = df['og']
df = df.drop(['Unnamed: 0','id'], axis=1)
# print(df.head())
# for col in df.columns:
#     print(col)
# y = df['t']
train, test = train_test_split(df, test_size=0.1, random_state=42)
# print(train)
# print(test)
test_ds = tf.data.Dataset.from_tensor_slices(test)
train_ds = tf.data.Dataset.from_tensor_slices(train)
# print(test_ds)
# print(train_ds)
def map_func(x):
  x1 = x[0]
  x2 = x[1]
  print(x1, x2)
  return (x1,x2,)
train_ds = train_ds.map(map_func)
test_ds = test_ds.map(map_func)
print("****",train_ds)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
itr_tf = train_ds.as_numpy_iterator()
for ele in itr_tf:
    print("!!!!!", ele, type(ele))
    break
# for sh_examples, en_examples in train_ds.batch(3).take(1):
#   print('> Examples in Shakespeare:')
#   for sh in sh_examples.numpy():
#     print(sh)
#   print()

#   print('> Examples in English:')
#   for en in en_examples.numpy():
#     print(en)

Tensor("strided_slice:0", shape=(), dtype=string) Tensor("strided_slice_1:0", shape=(), dtype=string)
Tensor("strided_slice:0", shape=(), dtype=string) Tensor("strided_slice_1:0", shape=(), dtype=string)
**** <_MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>
!!!!! (b' When day\xe2\x80\x99s oppression is not eased by night,  But day by night and night by day oppressed? ', b'When the pressure of the day is not relieved at night,But day is oppressed by night and night by day?') <class 'tuple'>


In [ ]:
MAX_TOKENS=128
BATCH_SIZE=64
BUFFER_SIZE=20_000
def prepare_batch(shk,en):

    shk =tokenizers.shk.tokenize(shk)     # Output is ragged.
    shk = shk[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    shk = shk.to_tensor()  # Convert to 0-padded dense Tensor

    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)]
    en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
    en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens

    return (shk, en_inputs), en_labels

def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
  )

print(tokenizers.en.get_vocab_size().numpy(), tokenizers.shk.get_vocab_size().numpy())

5638 5968


In [ ]:
train_batches = make_batches(train_ds)
val_batches = make_batches(test_ds)

In [ ]:
for (shk, en), en_labels in train_batches.take(1):
  break

print(shk.shape)
print(en.shape)
print(en_labels.shape)

(64, 56)
(64, 59)
(64, 59)


In [ ]:
for ele in train_batches.take(1).as_numpy_iterator():
    print(ele)
    break

((array([[   2,   64,   53, ...,    0,    0,    0],
       [   2,   23, 1451, ...,    0,    0,    0],
       [   2,   75, 2758, ...,    0,    0,    0],
       ...,
       [   2,   83, 2390, ...,    0,    0,    0],
       [   2,  110,  220, ...,    0,    0,    0],
       [   2,   57,   31, ...,    0,    0,    0]]), array([[  2,  83,  60, ...,   0,   0,   0],
       [  2, 101, 117, ...,   0,   0,   0],
       [  2,  85, 117, ...,   0,   0,   0],
       ...,
       [  2, 177,  96, ...,   0,   0,   0],
       [  2, 106,   7, ...,   0,   0,   0],
       [  2,  62,  35, ...,   0,   0,   0]])), array([[  83,   60,  303, ...,    0,    0,    0],
       [ 101,  117,   99, ...,    0,    0,    0],
       [  85,  117, 2850, ...,    0,    0,    0],
       ...,
       [ 177,   96,   83, ...,    0,    0,    0],
       [ 106,    7,   45, ...,    0,    0,    0],
       [  62,   35,  178, ...,    0,    0,    0]]))


In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):#encoding + embedding
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [ ]:
# embed_shk = PositionalEmbedding(vocab_size=40_000 + 1, d_model=16)
# embed_en = PositionalEmbedding(vocab_size=40_000 + 1, d_model=16)

# shk_emb = embed_shk(vec_x_train)
# en_emb = embed_en(vec_y_train)

In [ ]:
# en_emb._keras_mask

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)


    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]


  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)


    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    # print(list(inputs.as_numpy_iterator()))
    context, x = inputs
    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:
transformer = Transformer(
    num_layers=4,
    d_model=256,
    num_heads=8,
    dff=512,
    input_vocab_size=tokenizers.shk.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    dropout_rate=0.2)


In [ ]:
# example = ""
# for batch in val_batches:
#     # print(batch, "***********************")
#     example = batch
#     print(example)
#     break


In [ ]:
# transformer.build((2,100))
output = transformer((shk,en))

In [ ]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  10997760  
                                                                 
 decoder (Decoder)           multiple                  19329536  
                                                                 
 dense_16 (Dense)            multiple                  1448966   
                                                                 
Total params: 31776262 (121.22 MB)
Trainable params: 31776262 (121.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model=128, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(256)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)


In [ ]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy]
)


In [ ]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  10997760  
                                                                 
 decoder (Decoder)           multiple                  19329536  
                                                                 
 dense_16 (Dense)            multiple                  1448966   
                                                                 
Total params: 31776262 (121.22 MB)
Trainable params: 31776262 (121.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
transformer.fit(train_batches,
                epochs=25,
                validation_data=val_batches)
# 20 + 5

Epoch 1/25
729/729 [==============================] - 311s 425ms/step - loss: 4.5711 - masked_accuracy: 0.2721 - val_loss: 4.0890 - val_masked_accuracy: 0.3253
Epoch 2/25
729/729 [==============================] - 306s 419ms/step - loss: 3.9385 - masked_accuracy: 0.3364 - val_loss: 3.6222 - val_masked_accuracy: 0.3776
Epoch 3/25
729/729 [==============================] - 305s 418ms/step - loss: 3.5471 - masked_accuracy: 0.3781 - val_loss: 3.3729 - val_masked_accuracy: 0.4063
Epoch 4/25
729/729 [==============================] - 306s 419ms/step - loss: 3.2840 - masked_accuracy: 0.4067 - val_loss: 3.1413 - val_masked_accuracy: 0.4351
Epoch 5/25
729/729 [==============================] - 307s 420ms/step - loss: 3.1120 - masked_accuracy: 0.4245 - val_loss: 3.0190 - val_masked_accuracy: 0.4512
Epoch 6/25
729/729 [==============================] - 305s 418ms/step - loss: 2.9096 - masked_accuracy: 0.4480 - val_loss: 2.9031 - val_masked_accuracy: 0.4656
Epoch 7/25
729/729 [====================

In [ ]:
transformer.save_weights('drive/MyDrive/my_model/transformer-256-16')


In [ ]:
class Translator(tf.Module):
  def __init__(self, tokenizers, transformer):
    self.tokenizers = tokenizers
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_TOKENS):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    sentence = self.tokenizers.shk.tokenize(sentence).to_tensor()

    encoder_input = sentence

    # As the output language is English, initialize the output with the
    # English `[START]` token.
    start_end = self.tokenizers.en.tokenize([''])[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`.
    text = tokenizers.en.detokenize(output)[0]  # Shape: `()`.

    tokens = tokenizers.en.lookup(output)[0]

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)

    return text, tokens

In [ ]:
translator = Translator(tokenizers, transformer)


In [ ]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
  print(f'{"Ground truth":15s}: {ground_truth}')


In [ ]:
sentence = 'None but the king?'
ground_truth = 'Only the king?'

translated_text, translated_tokens = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)


Input:         : None but the king?
Prediction     : nothing but the king ?
Ground truth   : Only the king?


In [ ]:
sentence = ' A sample to the youngest, to the more mature  A glass that feated them, and to the graver  A child that guided dotards; '
ground_truth = 'He was an example to the youngest, to the full-grown a model for their own behavior, and seemed to serious observers like a child leading old people.'

translated_text, translated_tokens = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)


Input:         :  A sample to the youngest, to the more mature  A glass that feated them, and to the graver  A child that guided dotards; 
Prediction     : a sam - faced to the youngest daughter , to the more appropriate mirror that gave birth to a child who slapseed a child .
Ground truth   : He was an example to the youngest, to the full-grown a model for their own behavior, and seemed to serious observers like a child leading old people.


In [ ]:
sentence = 'For you, Posthumus,  So soon as I can win the offended king,  I will be known your advocate: '
ground_truth = 'As for you, Posthumus, as soon as I can calm the upset king, I will speak in your defense.'

translated_text, translated_tokens = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

Input:         : For you, Posthumus,  So soon as I can win the offended king,  I will be known your advocate: 
Prediction     : you , posthumus , as soon as i can win the offend king , i will be known as your servant .
Ground truth   : As for you, Posthumus, as soon as I can calm the upset king, I will speak in your defense.


In [ ]:
sentence = 'And as for thee my king, i pray that thou may be victorious in thy attempts'
ground_truth = ""

translated_text, translated_tokens = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

Input:         : And as for thee my king, i pray that thou may be victorious in thy attempts
Prediction     : and as for you , my king , i beg that you can be victorious in your attempt .
Ground truth   : 
